<a href="https://colab.research.google.com/github/Rajaguhan437/Custom_GAN_to_CLIP/blob/main/GAN_CLIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cloning into 'CLIP'...
remote: Enumerating objects: 243, done.
remote: Total 243 (delta 0), reused 0 (delta 0), pack-reused 243
Receiving objects: 100% (243/243), 8.92 MiB | 8.10 MiB/s, done.
Resolving deltas: 100% (123/123), done.


Cloning into 'taming-transformers'...
remote: Enumerating objects: 1339, done.
remote: Total 1339 (delta 0), reused 0 (delta 0), pack-reused 1339
Receiving objects: 100% (1339/1339), 409.77 MiB | 17.69 MiB/s, done.
Resolving deltas: 100% (279/279), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
## Downloading Libraries and dependenices

!pip install --no-deps ftfy regex tqdm
!pip install omegaconf==2.0.0 pytorch-lightning==1.0.0
!pip install einops

# omegaconf ==> to load config files

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pytorch_lightning-1.0.0-py3-none-any.whl (510 kB)
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 2.0.2
    Uninstalling pytorch-lightning-2.0.2:
      Successfully uninstalled pytorch-lightning-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
taming-transformers 0.0.1 requires pytorch-lightning>=1.0.8, but you have pytorch-lightning 1.0.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
## importing libraries

import numpy as np
import torch, os, imageio, pdb, math
import torchvision
import torchvision.transforms as T
import torchvision.transforms.functional as TF

import PIL
import matplotlib.pyplot as plt

import yaml
from omegaconf import OmegaConf


# import warnings
# warnings.filterwarnings('ignore')


In [5]:
def show_from_tensor(tensor):
  img = tensor.clone()
  img = img.mul(255).byte()
  img = img.cpu().numpy().transpose((1, 2, 0))

  plt.figure(figsize=(10,7))
  plt.axis('off')
  plt.imshow(img)
  plt.show()

def norm_data(data):
  return (data.clip(-1, 1)+1)/2  ### range between 0 and 1 

In [6]:
### hyperparameters

lr = .5
batch_size = 1
weight_decay = .1
noise_factor = .1
device = "cuda"

total_iter = 100
im_shape = [225, 400, 3] # height, width, channel
size1, size2, channels = im_shape


In [7]:
### CLIP Model(connects texts with images) ==> Download

!git clone https://github.com/openai/CLIP.git

'''
# CLIP module is not in python path. So, 
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/CLIP')
from CLIP import clip
'''
#!pip install CLIP
# from CLIP import Clip  ==> wrong way of import
from CLIP import clip # correct way

Cloning into 'CLIP'...
remote: Enumerating objects: 243, done.
remote: Total 243 (delta 0), reused 0 (delta 0), pack-reused 243
Receiving objects: 100% (243/243), 8.92 MiB | 22.06 MiB/s, done.
Resolving deltas: 100% (124/124), done.


In [8]:
### CLIP Model ==> Instantiation 

clipmodel, preprocess = clip.load("ViT-B/32", device=device, jit=False)
clipmodel.eval()
print(clip.available_models())

print("Clip model visual input resolution:", clipmodel.visual.input_resolution)

device = torch.device("cuda:0")
torch.cuda.empty_cache()

100%|███████████████████████████████████████| 338M/338M [00:05<00:00, 61.9MiB/s]


['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']
Clip model visual input resolution: 224


In [9]:
### Taming Transformer ==> Download 

!git clone https://github.com/CompVis/taming-transformers.git

### VQGan Arch ==> Download

%cd /content/taming-transformers/

!mkdir -p models/vqgan_imagenet_f16_16384/chkpts
!mkdir -p models/vqgan_imagenet_f16_16384/configs

if len(os.listdir("models/vqgan_imagenet_f16_16384/chkpts/")) == 0:
  !wget "https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fckpts%2Flast.ckpt&dl=1" -O "models/vqgan_imagenet_f16_16384/chkpts/last.ckpt"  

if len(os.listdir("models/vqgan_imagenet_f16_16384/configs/")) == 0:
  !wget "https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fconfigs%2Fmodel.yaml&dl=1" -O "models/vqgan_imagenet_f16_16384/configs/model.yaml"  

!pip uninstall torchtext --yes # otherwise results in error during from taming.models.vqgan import VQModel
!pip install --upgrade taming-transformers # otherwise results in torch._six import error

Cloning into 'taming-transformers'...
remote: Enumerating objects: 1339, done.
remote: Total 1339 (delta 0), reused 0 (delta 0), pack-reused 1339
Receiving objects: 100% (1339/1339), 409.77 MiB | 25.64 MiB/s, done.
Resolving deltas: 100% (279/279), done.
/content/taming-transformers
--2023-05-02 10:15:06--  https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fckpts%2Flast.ckpt&dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/c8b3c54d-118e-43fc-a137-641e32802508/last.ckpt [following]
--2023-05-02 10:15:09--  https://heibox.uni-heidelberg.de/seafhttp/files/c8b3c54d-118e-43fc-a137-641e32802508/last.ckpt
Reusing existing connection to heibox.uni-heidelberg.de:443.
HTTP request sent, awaiting response... 200 OK
Length: 980092370 (935M) [a

In [14]:
### taming-transformer(VQGan) Model ==> Instantiation 

from taming.models.vqgan import VQModel

def load_config(config_path, display=False):
  config_data = OmegaConf.load(config_path)
  if display:
    print(yaml.dump(OmegaConf.to_container(config_data)))
  return config_data

def load_vqgan(config, chk_path=None):
  model = VQModel(**config.model.params)
  if chk_path is not None:
    state_dict = torch.load(chk_path, map_location='cpu')["state_dict"]
    missing, unexpected = model.load_state_dict(state_dict, strict=False)
  return model.eval()

def generator(x):
  x = taming_model.post_quant_conv(x)
  x = taming_model.decoder(x)
  return x

taming_config = load_config("./models/vqgan_imagenet_f16_16384/configs/model.yaml", display=True)
taming_model = load_vqgan(taming_config, chk_path="./models/vqgan_imagenet_f16_16384/chkpts/last.ckpt")

model:
  base_learning_rate: 4.5e-06
  params:
    ddconfig:
      attn_resolutions:
      - 16
      ch: 128
      ch_mult:
      - 1
      - 1
      - 2
      - 2
      - 4
      double_z: false
      dropout: 0.0
      in_channels: 3
      num_res_blocks: 2
      out_ch: 3
      resolution: 256
      z_channels: 256
    embed_dim: 256
    lossconfig:
      params:
        codebook_weight: 1.0
        disc_conditional: false
        disc_in_channels: 3
        disc_num_layers: 2
        disc_start: 0
        disc_weight: 0.75
      target: taming.modules.losses.vqperceptual.VQLPIPSWithDiscriminator
    monitor: val/rec_loss
    n_embed: 16384
  target: taming.models.vqgan.VQModel

Working with z of shape (1, 256, 16, 16) = 65536 dimensions.


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:12<00:00, 45.8MB/s]


8.19kB [00:00, 140kB/s]                    


loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth
VQLPIPSWithDiscriminator running with hinge loss.


In [39]:
### values that we are going to optimize to create beautiful images

class Parameters(torch.nn.Module):
  def __init__(self):
    super(Parameters, self).__init__()
    self.data = .5*torch.randn(batch_size, 256, size1//16, size2//16).cuda() # 1 x 256 x 14 x 25
    self.data = torch.nn.Parameter(torch.sin(self.data))

  def forward(self):
    return self.data

def init_params():
  params = Parameters().cuda()
  optimizer = torch.optim.Adam([{'params':[params.data], 'lr':lr }], weight_decay = weight_decay)
  return params, optimizer

In [36]:
### Encodings the text

normalize = torchvision.transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))

def encodeText(text):
  t = clip.tokenize(text).cuda()
  t = clipmodel.encode_text(t).detach().clone()
  return t

def createEncodings(include, exclude, extras):
  include_enc = []
  for text in include:
    include_enc.append(encodeText(text))

  exclude_enc = encodeText(exclude) if exclude != '' else 0
  extras_enc = encodeText(extras) if exclude != '' else 0

  return include_enc, exclude_enc, extras_enc


In [37]:
### Image preprocess

augTransform = torch.nn.Sequential(
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomAffine(30, (.2, .2), fill=0)
).cuda()

In [38]:
### Initiallize

Params,  optimizer = init_params()

with torch.no_grad():
  print(Params().shape)
  img = norm_data(generator(Params()).cpu()) # 1 x 3 x 224 x 400 [225 x 400]
  print("img dimensions:", img.shape)
  show_from_tensor(img[0])

  

NameError: ignored